# ragas评估

## rag pipline

In [1]:
import warnings
warnings.filterwarnings('ignore')
from model import RagEmbedding, RagLLM, QwenLLM
from langchain_chroma import Chroma
import chromadb

In [2]:
sys_prompt = "你是企业员工助手，熟悉公司考勤和报销标准等规章制度，需要根据提供的上下文信息context来回答员工的提问。\
请直接回答问题，如果上下文信息context没有和问题相关的信息，请直接回答[不知道,请咨询HR] "
llm = RagLLM(sys_prompt=sys_prompt)

In [3]:
# 加载向量库
embedding_model = RagEmbedding()

d:\Desktop\LLM资料\Proj\ChatDoc_RagLangChain\ChatDoc1.0\model.py:99: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedding = HuggingFaceEmbeddings(model_name=model_path,


In [4]:
# chroma_client = chromadb.HttpClient(host="localhost", port=8000)
chroma_client = chromadb.PersistentClient(path="./data/data_base")

In [5]:
zhidu_db = Chroma("zhidu_db",
                  embedding_model.get_embedding_fun(),
                  client=chroma_client)

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_template = """
问题：{question}
"{context}"
回答：
"""

def run_rag_pipline_without_stream(query, k=3):
    related_docs = zhidu_db.similarity_search(query, k=k)
    context_list = [f"上下文{i+1}: {doc.page_content} \n" \
                         for i, doc in enumerate(related_docs)]
    context = "\n".join(context_list)

    prompt = PromptTemplate(  # 创建提示模板
        input_variables=["question", "context"],
        template=prompt_template, )  # 使用传入的模板（rule_template）
    llm_prompt = prompt.format(question=query, context=context)  # 格式化提示，填入查询和上下文

    response = llm(llm_prompt, stream=False)

    return response, context_list

## 构建评估数据集
- 问题
- 标准答案
- 上下文信息
- 生成的答案

In [7]:
questions = [
    "伙食补助费标准是什么?",
    "出差可以买意外保险吗？需要自己购买吗",
]
ground_truths = [
    "伙食补助费标准: 西藏、青海、新疆 120元/人、天 其他省份 100元/人、天",
    "出差可以购买交通意外保险，由单位统一购买，不再重复购买",
]

In [8]:
answers = []
contexts = []

for query in questions:
    response, context_list = run_rag_pipline_without_stream(query, k=3)
    answers.append(response)
    contexts.append(context_list)


In [9]:
from datasets import Dataset

In [10]:
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

In [11]:
dataset = Dataset.from_dict(data)

In [12]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from ragas import evaluate
from ragas import RunConfig

In [13]:
eval_llm = QwenLLM(sys_prompt=sys_prompt)
embedding_model = RagEmbedding()
eval_embedding_fn = embedding_model.get_embedding_fun()

In [14]:
config = RunConfig(timeout=1200, log_tenacity=True)

In [15]:
result = evaluate(
    dataset = dataset,
    llm=eval_llm,
    embeddings=eval_embedding_fn,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    raise_exceptions=True,
    run_config=config
)

df = result.to_pandas()

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

In [16]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,伙食补助费标准是什么?,[上下文1: <table><caption>伙食补助费参考以下标准：</caption>\...,伙食补助费的标准如下：\n\n- 西藏、青海、新疆：120 元/人、天\n- 其他省份：10...,伙食补助费标准: 西藏、青海、新疆 120元/人、天 其他省份 100元/人、天,1.0,1.0,1.00,0.960389
1,出差可以买意外保险吗？需要自己购买吗,[上下文1: 差旅费用标准\n差旅费开支范围包括工作人员临时到常驻地以外地区公务出差所发生的...,根据公司的差旅费用标准，乘坐飞机、火车、轮船等交通工具的，每人次可以购买交通意外保险一份。如...,出差可以购买交通意外保险，由单位统一购买，不再重复购买,1.0,1.0,0.75,0.937492
